# PS 3, Week 13: Bivariate Regression
Authors: Eric Van Dusen, Andrew Little, William McEachen

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
from ipywidgets import *
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from IPython.display import display, Markdown
%matplotlib inline

## Linear Models:
When we have two continuous variables (one dependent, one independent), we can use *bivariate regression* to determine how closely the two are related. Biviarate regression is used to determine how changes in one variable -- the independent variable, often denoted $X$ -- can predict changes in another, the dependent variable, often denoted $Y$. Bivariate regression relies on a linear model, which follows the form $ Y_i= \alpha + \beta X_i$, where $\alpha$ is the y-intercept and $\beta$ is the slope. 

If we assume that the relationship between our variables is not perfect (or, in the real world, if there is some predictable inaccuracy in our measurement), we add an error term $\epsilon_i$: $ Y_i= \alpha + \beta X_i + \epsilon_i$. 

## Motivating Ordinary Least Squares

To understand how we might create an equation for two variables, let's consider the relationship between health care spending and life expectancy

In [ ]:
spend = pd.read_csv('outspend2015.csv')
spend

Remember the US is a bit of an outlier in this relationship, so to make some of the key concepts clear we will drop the US.

In [ ]:
spend_noUS = spend[spend['Spending'] < 14]
spend_noUS

Like last week, we can make a scatterpot:

In [ ]:
sns.scatterplot(x='Spending', y='Expectancy', data=spend_noUS)

Above, it appears that as Spending increases (as we move further to the right on the x-axis), Expectancy also increases. If we wanted to use this data to make future predictions, we could use a linear model to represent the variables' relationship. Below, you can change the slope and intercept of the line to best fit the data:

In [ ]:
def draw_line(slope, intercept):
    #The Linear Model
    #    def f(x):
    #    return intercept*(slope-1)/30*x +intercept
    def f(x):
        return intercept + slope*x
    x = np.arange(4,16)
    y_pred = f(x)
    points = (zip(spend_noUS.Spending, spend_noUS.Expectancy))
    #The line
    sns.scatterplot(x='Spending', y='Expectancy', data=spend_noUS)
    plt.plot(x,y_pred)
    #The actual data 
    plt.xlabel('Spending')
    plt.ylabel('expectancy')
    display(Markdown(rf'$\hat y$= {slope}$X$ + {intercept}:'))
    
interact(draw_line, slope=(0.0,3), intercept=(65,80))


### What line is best?
When we are evaulating how "good" a line is, we must address the *residuals*, the difference between the real and predicted values of y: $u_i = Y_i - \hat{Y_i}$. Because every real y value has an associated residual, we need some way to aggregate the residuals if we are to measure the overall quality of a line

#### Absolute value
One measurement of loss is calculated by adding the absolute value of the residuals together:
$$\sum_{i=1}^n |u_i| = \sum_{i=1}^n |Y_i - \hat{Y_i}|$$

#### Squared error:
Another measurement is the *squared error*, calculated by adding the squared values of the residuals:
$$\sum_{i=1}^n |u_i^2| = \sum_{i=1}^n (Y_i-\hat{Y_i})^2$$

For either measurement, we want the line that results in the smallest value (indicating that the total difference between the predicted and actual values is small). Below, try to minimize either the absolute or squared loss:

In [ ]:
def draw_line(slope, intercept):
    #The Linear Model
    def f(x):
        return intercept + slope*x
    x = np.arange(4,16)
    y_pred = f(x)
    display(Markdown(rf'$\hat y$= {slope}$X$ + {intercept}:'))
    #The line
    plt.plot(x,y_pred)
    #The Data
    sns.scatterplot(x='Spending', y='Expectancy', data=spend_noUS)
    #Print the loss
    print("Square Residual Sum:", sum([(y-f(x))**2 for x,y in zip(spend_noUS.Spending, spend_noUS.Expectancy)]))
    print("Absolute Residual Sum:", sum([abs(y-f(x))for x, y in zip(spend_noUS.Spending, spend_noUS.Expectancy)]))
    
interact(draw_line, slope=(0.0,2), intercept=(65,80), continuous_update=False)

What's the smallest squared error/absolute error you can produce?

## Ordinary Least Squares
Statisticians prefer to use the line that minimizes the squared residuals. To find the slope ($\beta$) and y-intercept ($\alpha$), the following equations are used:
$$\beta = \frac{\sum_{i=1}^n (X_i - \overline{X})(Y_i - \overline{Y})}{\sum_{i=1}^n (X_i - \overline{X})^2}$$
$$\alpha = \overline{Y}-\beta\overline{X}$$
*Reminder*: $\overline{X}$ represents the mean value of X.

### Using Python:
To calculate the slope and y-intercept for the linear model of X and Y, use <code>stats.linregress(X, Y)</code>. This returns a LinregressResult which holds the slope, intercept, the associated r and p values, and standard error (we'll cover what those mean next). To access the slope and intercept, you can index into the LinregressResult, as shown below:

In [ ]:
spend_expect_result = stats.linregress(spend_noUS.Spending, spend_noUS.Expectancy)
spend_expect_result

In a few cells we will want to plot a line with this slope and intercept on top of our data. Rather than copy and paste, we can "pull" the results from our regression with the following code:

In [ ]:
lm_slope = spend_expect_result[0]
lm_intercept = spend_expect_result[1]
print("slope: ", lm_slope, ", Intercept = ", lm_intercept)

Now that we have the slope and intercept, we can plot the line of best fit alongside the original data:

In [ ]:
sns.scatterplot(x='Spending', y='Expectancy', data=spend_noUS)
plt.plot(spend_noUS.Spending, spend_noUS.Spending*lm_slope + lm_intercept, label='OLS model')
plt.xlabel('Health Care Spending')
plt.ylabel('Life Expectancy')
plt.legend()

## Extrapolating to the Population
Once we have produced slope and intercept values for our model, we wish to know the potential applicability to the population at large. To do this, we build on our previous lectures about hypothesis testing.

Here is a bit more about how this is done than we will do in lectures. What matters in the end is knowing how to interpet the model output. 

To start, we must find the *variance of residuals*, or $\hat{\sigma}^2$. This statistic is a measurement of the general difference between predicted and actual values of Y:
$$\hat{\sigma}^2 = \frac{\sum_{i=1}^n u_i^2}{n-2}$$

From the formula above, we can see that $\hat{\sigma}^2$ increases as the size of the residuals increase and decreases as our sample size, n, increases. We can then calculate the variance and standard error for the estimate of the slope value using the formula below:
$$var(\beta) = \frac{\hat{\sigma}^2}{\sum_{i=1}^n (X_i-\overline{X})^2}$$  

$$se(\beta) = \sqrt{var(\beta)}$$

### The Null Hypothesis
Once we have the standard error, the procuedures to do hypothesis testing are exactly the same as with difference of means tests.

Our null hypothesis is that the slope is zero. So the t-statistic:
$$t_\beta = \frac{\beta}{se(\beta)}$$
tells us "how many standard errors away from zero is our esimate"

Once we have calculated the t-statistic, we can use an online table (or in the appendix of your textbook) to find the p-value for whether there is a statistically significant relationship between X and Y.

### Using Python
To find the p-value for the slope of our model, we use <code>stats.linregress(X, Y)</code>. As we saw earlier, this returns the standard error as well, so we can find that using the same function. To access either value, index into the fourth value of the result for the p value, and the fifth value for the standard error.

In [16]:
spend_expect_result

LinregressResult(slope=0.6882635096164225, intercept=77.22256835334136, rvalue=0.6461317839588818, pvalue=2.7586432526813464e-05, stderr=0.1415238870758501)

Let's pull out the standard error:

In [17]:
se_b = spend_expect_result[4]
se_b

0.1415238870758501

And use it to create a 95% confidence interval:

In [18]:
slope_lower = lm_slope - 1.96*se_b
slope_upper = lm_slope + 1.96*se_b
print("[", slope_lower, " , ", slope_upper, "]")

[ 0.4108766909477563  ,  0.9656503282850886 ]


### [OPTIONAL] Interactive Visual
Below, we plot the relationship between two random variables. What happens to the results of the OLS model as you change the covariance? When you change the randomness of Y?

In [19]:
def cloud(covariance, rand_factor):
    X,Y = list(zip(*np.random.multivariate_normal([5,5], [[1,covariance],[covariance,1]], size=1000).tolist()))
    Y = Y+np.random.random(1000)*rand_factor
    sns.scatterplot(x=X, y=Y)
    print(stats.linregress(X,Y))
    print("R-value:",stats.pearsonr(X,Y)[0])

interact(cloud, covariance=(-1.0,1.0), rand_factor=(0,10), continuous_update=False)

interactive(children=(FloatSlider(value=0.0, description='covariance', max=1.0, min=-1.0), IntSlider(value=5, …

<function __main__.cloud(covariance, rand_factor)>

## [OPTIONAL] Goodness of Fit with non-Linear models
Now that we have our linear model, we want to evaulate how well it tracks the relationship between the independent and dependent variables (X and Y). Below, which models are fit well by a linear model?

In [ ]:
#A Generic Plotting function for some f(x)
def plot_func(f, label):
    x = np.random.randint(1,25, size=50)
    y = f(x)
    sns.scatterplot(x=x, y=y, label=label)
    result = stats.linregress(x, y)
    slope = result[0]
    intercept = result[1]
    plt.plot(x, x * slope + intercept, label='OLS model')
    plt.legend()

In [ ]:
def f(x):
    return 2*x
plot_func(f, 'linear')

In [ ]:
def f(x): 
    return 4*x**2 + np.random.random(50)
plot_func(f, 'quadratic')

In [ ]:
def f(x):
    return  8*np.log(x) + np.random.random(50)
plot_func(f, 'logarithmic')

As we can see, linear models can better represent some relationships than others! We need some measure to quantify if a linear model is appropriate.

## $R^2$  Statistic
In order to determine if our linear model is a good fit, we need to understand how much of the variance in the values of Y can be captured in our model. In the 3 plots above, a linear model had differing abilities to represent that variation.  
*Review*: Which functions variation was best captured by a linear model?  

We need a measurement that compares the overall variation in Y values with the variation that is not represented by our OLS model. This second kind is the *residual variation* because it is the difference between true values of Y and those predicted by our model. The $R^2$ statistic is a value between 0 and 1 that represents the proportion of variation in our Y value that can be accounted for by our model:
$$R^2 = 1-\frac{\text{residual variation}}{\text{total variation}}$$
The residual variation, or Residual Sum of Squares (RSS), is calculated by adding all squared residuals between the predicted and true values of Y:
$$RSS = \sum_{i=1}^n u_i^2 = \sum_{i=1}^n (Y_i - \hat{Y_i})^2$$
The total variation, or Total Sum of Squares (TSS), is calculated by adding the squared differences between each value of Y and the mean value of Y:
$$TSS = \sum_{i=1}^n (Y_i - \overline{Y})^2$$
Therefore, the $R^2$ statistic can be found thus:
$$R^2 = 1 - \frac{RSS}{TSS} = 1 - \frac{\sum_{i=1}^n (Y_i - \hat{Y_i})^2}{\sum_{i=1}^n (Y_i - \overline{Y})^2}$$

There is another formula for calculating $R^2$ using the Model Sum of Squares (MSS). The Model Sum of Squares is the total squared difference between predicted values of Y and the average value of Y:
$$MSS = \sum_{i=1}^n (\hat{Y_i} - \overline{Y})^2$$
The formula for $R^2$ using MSS is:
$$R^2 = \frac{MSS}{TSS} = \frac{\sum_{i=1}^n (\hat{Y_i}-\overline{Y})^2}{\sum_{i=1}^n (Y_i - \overline{Y})^2}$$

### Using Python
There are 2 ways to calculate the $R^2$ value using Python. We will show both and demonstrate that they return the same result:  
1) Write functions calculating the RSS and TSS, and then use those to calculate the $R^2$ statistic (we have done this below  
2) Run <code>stats.linregres(X, Y)</code> and square the third value that it returns

In [24]:
#Method 1:
def rss(y, y_pred):
    """Return the Residual Sum of Squares between y and y_pred, the predicted values of y"""
    return sum((y - y_pred)**2)
def tss(y):
    """Return the Total Sum of Squares for y"""
    avg = np.mean(y)
    return sum((y - avg)**2)

def mss(y, y_pred):
    """Return the Model Sum of Squares for y and y_pred, the predicted values of y"""
    avg = np.mean(y)
    return sum((y_pred-avg)**2)

def r2(y, y_pred):
    """Return the R-squared statistic for y and the predicted values of y"""
    return 1 - (rss(y, y_pred) / tss(y))

predicted_y = spend_noUS.Spending*lm_slope + lm_intercept
r2(spend_noUS.Expectancy, predicted_y)

0.4174862822418868

In [25]:
#Method 2:
spend_expect_result[2]**2

0.4174862822418871

As we can see, the two results are the same! This is where the name R^2 comes from. Exactly why these end up being the same is beyond our scope; for a formal proof, follow [this link](https://economictheoryblog.com/2014/11/05/proof/).